In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

## External DB
### 1) **Tradable** instruments
- e.g.: `itemtype` may be ETF, stocks and etc.

In [2]:
itemtype = 'ETF'

#### Load external data formatted as plain files, and then pickle them.
- We drop any instrument if:
  - `min_datapoints`: # of data points <= 3-year long.

In [3]:
data_files = ['price_d', 'volume_d', 'price_w', 'volume_w', 'price_m', 'volume_m']
filepath = '../../data/external/'
df = {}
for filename in data_files:
    df[filename] = pd.read_csv(filepath + filename + '.dat', header=7)
    df[filename].drop(range(0,6), inplace=True)
    df[filename].rename(columns={'Code': 'date'}, inplace=True)
    df[filename].date = pd.to_datetime(df[filename].date)
    name_cols = df[filename].columns[1:]
    df[filename][name_cols] = df[filename][name_cols].apply(pd.to_numeric, errors='coerce', axis=1)


#### Merge data sets and melt them into one tall and thin data frame.
- `df_db[freq]` is your final data frame.
  - `freq` = {d|w|m} for different frequencies.

In [4]:
df_pr = {}
df_vol = {}
df_db = {}

frequency = ['d', 'w', 'm']
# We need an extra 1 record for return calcaultion.
min_datapoints = {'d': 365*3+1, 'w': 52*3+1, 'm': 12*3+1}
window_3m = {'d': 90, 'w': 12, 'm': 3}

for freq in frequency:
    pr = df['price_'+freq].dropna(thresh=min_datapoints[freq], axis=1).dropna()
    vol = df['volume_'+freq].dropna(thresh=min_datapoints[freq], axis=1).dropna()
    df_pr[freq] = pd.melt(pr, id_vars=['date'], var_name='itemcode', value_name='price')
    df_vol[freq] = pd.melt(vol, id_vars=['date'], var_name='itemcode', value_name='volume')
    df_db[freq]= pd.merge(df_pr[freq], df_vol[freq], left_on=['date', 'itemcode'], right_on=['date', 'itemcode'], how='outer')
    df_db[freq] = df_db[freq].assign(trading_amt_mln=(df_db[freq].price*df_db[freq].volume).divide(10**6).rolling(window_3m[freq], min_periods=1).mean())
    df_db[freq] = df_db[freq].assign(ret=np.log(1+df_db[freq].groupby('itemcode').price.pct_change()))
    df_db[freq]['itemtype'] = itemtype
#     df_db[freq].to_pickle(filepath + 'price_db_' + freq + '.pkl')

### 2) **Non-tradable** instruments
- e.g.: rates

In [7]:
df['ecos_w'] = pd.read_csv(filepath + 'ecos_w' + '.dat', header=3, parse_dates=['date'])
df['ecos_w']['itemcode'] = 'CALL'
df['ecos_w']['itemtype'] = 'riskfree'
df['ecos_w'].rename(columns={'call':'price'}, inplace=True)

In [8]:
df_db['w'] = pd.concat([df_db['w'], df['ecos_w']])

## Pickle final restuls

In [9]:
for freq in frequency:
    df_db[freq].to_pickle(filepath + 'price_db_' + freq + '.pkl')

In [10]:
for freq in frequency:
    display(df_db[freq])

,date,itemcode,price,volume,trading_amt_mln,ret,itemtype
0,2016-08-17,A069500,23459.0,2999724.0,70370.525316,NaN,ETF
1,2016-08-18,A069500,23629.0,3610366.0,77839.931765,0.007221,ETF
2,2016-08-19,A069500,23684.0,3608250.0,80379.218843,0.002325,ETF
3,2016-08-22,A069500,23546.0,2882931.0,77254.787464,-0.005844,ETF
4,2016-08-23,A069500,23670.0,1943488.0,71004.302163,0.005252,ETF
...,...,...,...,...,...,...,...
26803,2021-03-02,A245340,17800.0,15862.0,383.351191,0.003940,ETF
26804,2021-03-03,A245340,17785.0,13398.0,383.138036,-0.000843,ETF
26805,2021-03-04,A245340,17685.0,18258.0,385.770184,-0.005639,ETF
26806,2021-03-05,A245340,17520.0,13537.0,386.990621,-0.009374,ETF


,date,itemcode,price,volume,trading_amt_mln,ret,itemtype
0,2017-12-15,A069500,30640.0000,8519096.0,261025.101440,NaN,ETF
1,2017-12-22,A069500,30152.0000,8482482.0,258394.449352,-0.016055,ETF
2,2017-12-28,A069500,30762.0000,7560958.0,249793.029567,0.020029,ETF
3,2018-01-05,A069500,31203.0000,7465878.0,245584.219983,0.014234,ETF
4,2018-01-12,A069500,30982.0000,5001713.0,227459.990420,-0.007108,ETF
...,...,...,...,...,...,...,...
736,2021-02-10,CALL,0.0049,NaN,NaN,NaN,riskfree
737,2021-02-19,CALL,0.0051,NaN,NaN,NaN,riskfree
738,2021-02-26,CALL,0.0059,NaN,NaN,NaN,riskfree
739,2021-03-05,CALL,0.0047,NaN,NaN,NaN,riskfree


,date,itemcode,price,volume,trading_amt_mln,ret,itemtype
0,2018-03-30,A069500,29992.0,8620284.0,258539.557728,NaN,ETF
1,2018-04-30,A069500,30750.0,4494562.0,198373.669614,0.024959,ETF
2,2018-05-31,A069500,29607.0,18580502.0,315620.087314,-0.037879,ETF
3,2018-06-29,A069500,28603.0,14304926.0,365828.167531,-0.034499,ETF
4,2018-07-31,A069500,28436.0,5386124.0,370812.181052,-0.005856,ETF
...,...,...,...,...,...,...,...
1179,2020-11-30,A245340,16414.0,20275.0,279.354700,0.100741,ETF
1180,2020-12-30,A245340,16615.0,29841.0,375.580971,0.012171,ETF
1181,2021-01-29,A245340,17005.0,24363.0,414.298293,0.023202,ETF
1182,2021-02-26,A245340,17730.0,29283.0,476.429540,0.041751,ETF
